# Frozen Molecules and redefine a center to move/rotate Molecules:
**Task:** 
1. Choose any Molecule to be FROZEN (fixed) and it can NOT be moved/rotated
2. Redefine a different 3D point as a center for our cluster

## Iodine and water pentamer [$I$($H_{2}O$)$_6$]:
Cartesian coordinates [Angstrom]:

    I    0.00000000	     0.00000000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000
    O    0.00000000	     0.00000000	     0.00000000
    H    0.58708000	     0.75754000	     0.00000000
    H   -0.58708000	     0.75754000	     0.00000000

> **_NOTE:_**   
> `translate` and `rotate` methods return a brand new Molecule object

In [ ]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# importing de Molecule Class
from src.base_molecule import Molecule, Cluster

In [ ]:
# let's create a cluster with five (5) water molecules and one atom (Iodine)
water_molecule=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
iodine =[("I", 0, 0, 0)]

# if we want an accurate orden in our cluster, we should use COMMA 
# for example, Iodine as the first molecule in our cluster
w5_I = Cluster(Molecule(iodine) + 5 * Molecule(water_molecule))


# ALWAYS, you have SIX (6) individual molecules/atoms in your 'w5_I' cluster
print(f"Number of individual molecules/atoms: {w5_I.total_molecules}")
print(w5_I.xyz)


In [ ]:
from copy import deepcopy


# creating a brand new copy when 'I' will NOT be moved or rotated 
w5 = deepcopy(w5_I)

# let's redefine a new center for our sphere
new_x, new_y, new_z = 20, 20, 20
w5.sphere_center = (new_x, new_y, new_z)

# let's add another atoms (any) to show the new sphere center
w5 = w5.add_molecule([("F", new_x, new_y, new_z)])

# define the size for our sphere (boundary conditions)
sphere_radius = 10

# setting a sphere radius for the Cluster spherical boundary conditions
w5.sphere_radius = sphere_radius

# frozen molecule to avoid be moved or rotated (to show origin and the sphere)
w5.frozen_molecule = [0, 6]

print("sphere radius: ", w5.sphere_radius)
print("sphere center: ",  w5.sphere_center)

print(w5.xyz)

In [ ]:
from numpy import random
random_gen = random.default_rng(1234)

# to save snapshot and show as a movie
w5_xyz = ""

total_steps = 1000

# max step size to move a molecule
step = sphere_radius

for i in range(total_steps):
    # molecule to be selected between [0, w.total_molecules]
    # frozen molecules will not be moved or rotated
    mol = random_gen.choice(w5.total_molecules)

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-step, +step]
    tx = step * (random_gen.uniform() - 0.5)
    ty = step * (random_gen.uniform() - 0.5)
    tz = step * (random_gen.uniform() - 0.5)

    # saving coordinates as a string
    # w5_xyz += w5.xyz
    if i % 100 == 0: 
        w5_xyz += w5.xyz

    w5 = w5.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )
    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )

# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(w5.xyz)

!pip install py3Dmol
import py3Dmol

In [ ]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

In [ ]:
# 20% to check visually that everything is inside
sr = w5.sphere_radius * 1.2

# sphere center
cx, cy, cz = w5.sphere_center
sc = {"x": cx, "y": cy, "z": cz}

# axes
x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

# starting visualization
xyz_view = py3Dmol.view(width=700, height=500)#, linked=False, viewergrid=(2,2))
xyz_view.addModelsAsFrames(w5_xyz,'xyz')
# xyz_view.setStyle({'stick': {}})
xyz_view.setStyle({'sphere': {'radius': 0.8}})
xyz_view.addSphere({'center': sc, 
        'radius': sr, 
        'color' :'yellow',
        'alpha': 0.5,
        })

# animation base on several XYZ coordinates snapshots
xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

# cartesian 3D axes
xyz_view.addLine(x_axis)
xyz_view.addLine(y_axis)
xyz_view.addLine(z_axis)

xyz_view.addLabel("x", {
        'position':x_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("y", {
        'position':y_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("z", {
        'position':z_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })

xyz_view.zoomTo()
xyz_view.show()


# A better example...

## Why does someone need to use a sphere not in the origin? And freeze some atoms/molecules?

Well, ma lot of applications for that, 
- localize explorations around some groups in large molecules
- molecular docking
- a more exhaustive exploration
- ...

In [ ]:
from data.molecules_coordinates import metal_complex 

complex = Cluster(metal_complex, 10 * Cluster(water_molecule))

# setting a sphere radius for the Cluster spherical boundary conditions
# define the size for our sphere (boundary conditions) in Angstrom
complex.sphere_radius = 5

complex.sphere_center = (5, 5, 5)

# frozen molecule to avoid be moved or rotated (just water molecules dancing)
complex.frozen_molecule = [0]

print(complex)

In [ ]:
from numpy import random
random_gen = random.default_rng(1234)

# to save snapshot and show as a movie
complex_xyz = ""

total_steps = 1000

for i in range(total_steps):
    # molecule to be selected between [0, w.total_molecules]
    # frozen molecules will not be moved or rotated
    mol = random_gen.choice(complex.total_molecules)

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-step, +step]
    step = sphere_radius / 2
    tx = step * (random_gen.uniform() - 0.5)
    ty = step * (random_gen.uniform() - 0.5)
    tz = step * (random_gen.uniform() - 0.5)

    # saving coordinates as a string
    # complex_xyz += complex.xyz
    if i % 10 == 0: 
        complex_xyz += complex.xyz

    complex = complex.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )
    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )

# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(complex.xyz)

In [ ]:
# 20% to check visually that everything is inside
sr = complex.sphere_radius * 1.2

# sphere center
cx, cy, cz = complex.sphere_center
sc = {"x": cx, "y": cy, "z": cz}

# axes
x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

# starting visualization
xyz_view = py3Dmol.view(width=1000, height=800)#, linked=False, viewergrid=(2,2))
xyz_view.addModelsAsFrames(complex_xyz,'xyz')
xyz_view.setStyle({'stick': {}})
# xyz_view.setStyle({'sphere': {'radius': 0.8}})
xyz_view.addSphere({'center': sc, 
        'radius': sr, 
        'color' :'yellow',
        'alpha': 0.5,
        })

# animation base on several XYZ coordinates snapshots
xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

# cartesian 3D axes
xyz_view.addLine(x_axis)
xyz_view.addLine(y_axis)
xyz_view.addLine(z_axis)

xyz_view.addLabel("x", {
        'position':x_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("y", {
        'position':y_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("z", {
        'position':z_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })

xyz_view.zoomTo()
xyz_view.show()